In [ ]:
from pathlib import Path
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Определение класса Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256 * 8 * 8),
            nn.BatchNorm1d(256 * 8 * 8),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (256, 8, 8)),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)
    
# Загрузка сохраненных параметров модели
checkpoint = torch.load('model.pth')
img_channels = 3
latent_dim = 100

# Создание и загрузка генератора
generator = Generator(latent_dim, img_channels)
generator.load_state_dict(checkpoint['generator_state_dict'])
generator.eval()

def generate_images(generator, output_folder, num_images=1000):
    # Создаем папку, если ее нет
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)

    # Генерация и сохранение изображений
    for i in range(num_images):
        with torch.no_grad():
            z = torch.randn(1, latent_dim)
            generated_image = generator(z)

        # Преобразование тензора в изображение и сохранение
        generated_image_pil = transforms.ToPILImage()(generated_image.squeeze())
        generated_image_pil.save(output_path / f"generated_image_{i}.png")

output_folder = "./generated_images"
generate_images(generator, output_folder, num_images=1000)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.datasets import ImageFolder

# Определение классификатора
class Classifier(nn.Module):
    def __init__(self, img_channels):
        super(Classifier, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

root_folder = "./generated_images"
transform = Compose([
    Resize((64, 64)),
    ToTensor()
])

classifier_dataset = ImageFolder(root=root_folder, transform=transform)
classifier_dataloader = DataLoader(classifier_dataset, batch_size=128, shuffle=True)

# Определение и обучение классификатора
img_channels = 3
classifier = Classifier(img_channels)
optimizer_classifier = optim.Adam(classifier.parameters(), lr=0.001)
criterion_classifier = nn.BCELoss()

epochs_classifier = 10
for epoch in range(epochs_classifier):
    for batch_img, batch_label in classifier_dataloader:
        optimizer_classifier.zero_grad()
        output = classifier(batch_img)
        
        # Уменьшаем размерность и преобразуем тип данных целевой переменной
        batch_label = batch_label.unsqueeze(1).float()

        loss_classifier = criterion_classifier(output, batch_label)
        loss_classifier.backward()
        optimizer_classifier.step()

# Сохранение параметров классификатора
torch.save({'classifier_state_dict': classifier.state_dict()}, 'classifier.pth')
